In [8]:
import pytesseract
from PIL import Image
print(pytesseract.image_to_string('/Volumes/Sparsh SD/Semesters/sem 8/HMI/Mnist-Digit-Reader--master/gfg_dummy_pic.png'))


In [14]:
import tensorflow as tf
import tensorflow_datasets
import keras
from keras.datasets import mnist
import requests
requests.packages.urllib3.disable_warnings()
import ssl

In [61]:
#(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
#(x_train, y_train), (x_test, y_test) = tensorflow_datasets.load('mnist')
#test, train = tensorflow_datasets.load('mnist') 
(ds_train, ds_test), ds_info = tensorflow_datasets.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)


In [62]:
def normalize_img(image, label):
  """Normalizes images: `uint8` -> `float32`."""
  return tf.cast(image, tf.float32) / 255., label

ds_train = ds_train.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_train = ds_train.cache()
ds_train = ds_train.shuffle(ds_info.splits['train'].num_examples)
ds_train = ds_train.batch(128)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)


In [63]:
ds_test = ds_test.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_test = ds_test.batch(128)
ds_test = ds_test.cache()
ds_test = ds_test.prefetch(tf.data.AUTOTUNE)

In [64]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10)
])
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

model.fit(
    ds_train,
    epochs=6,
    validation_data=ds_test,
)

Epoch 1/6
469/469 [==============================] - 11s 6ms/step - loss: 0.3622 - sparse_categorical_accuracy: 0.9012 - val_loss: 0.2017 - val_sparse_categorical_accuracy: 0.9434
Epoch 2/6
469/469 [==============================] - 3s 7ms/step - loss: 0.1691 - sparse_categorical_accuracy: 0.9511 - val_loss: 0.1397 - val_sparse_categorical_accuracy: 0.9612
Epoch 3/6
469/469 [==============================] - 3s 7ms/step - loss: 0.1206 - sparse_categorical_accuracy: 0.9655 - val_loss: 0.1180 - val_sparse_categorical_accuracy: 0.9667
Epoch 4/6
469/469 [==============================] - 4s 9ms/step - loss: 0.0945 - sparse_categorical_accuracy: 0.9730 - val_loss: 0.0960 - val_sparse_categorical_accuracy: 0.9717
Epoch 5/6
469/469 [==============================] - 3s 6ms/step - loss: 0.0765 - sparse_categorical_accuracy: 0.9777 - val_loss: 0.0956 - val_sparse_categorical_accuracy: 0.9706
Epoch 6/6
469/469 [==============================] - 3s 6ms/step - loss: 0.0638 - sparse_categorical_acc

In [66]:
model.evaluate(ds_test)

79/79 [==============================] - 1s 5ms/step - loss: 0.0866 - sparse_categorical_accuracy: 0.9744


[0.08658287674188614, 0.974399983882904]

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

In [73]:
import cv2
import numpy as np

In [89]:
image = cv2.imread('/Volumes/Sparsh SD/Semesters/sem 8/HMI/Mnist-Digit-Reader--master/gfg_dummy_pic2.png')
img = cv2.resize(image, (28,28))
image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#pred = model.predict('/')

In [90]:
img = (np.expand_dims(image,0))

In [91]:
print(img.shape)

(1, 28, 28)


In [99]:
pred = model.predict(img).tolist()
pred = pred[0]
predicted_number = pred.index(max(pred))
predicted_number

3

In [110]:
import pickle
filename = 'handwriting_detector.pkl'

pickle.dump(model, open(filename, 'wb'))

OSError: [Errno 30] Read-only file system: '/handwriting_detector.pkl'

In [ ]:
loaded_model = pickle.load(open(filename, 'rb'))